In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > output.log 2>&1 &
!ollama pull phi4

In [ ]:
!pip install langchain faiss-cpu PyMuPDF requests langchain-community

In [ ]:
# 📦 All-in-One PDF QA with Phi-4, FAISS, and LangChain

import requests
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOllama
from langchain.chains import RetrievalQA
import os

# 1. 📥 Download sample PDF
pdf_url = "https://arxiv.org/pdf/2306.15595.pdf"  # Replace with your preferred PDF URL
pdf_path = "sample.pdf"
if not os.path.exists(pdf_path):
    response = requests.get(pdf_url)
    with open(pdf_path, 'wb') as f:
        f.write(response.content)

# 2. 📄 Load and split PDF
loader = PyMuPDFLoader(pdf_path)
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)

# 3. 🧠 Embeddings + FAISS
embeddings = OllamaEmbeddings(model="phi4")
vectorstore = FAISS.from_documents(chunks, embeddings)

# 4. 💬 Retrieval-based QA
llm = ChatOllama(model="phi4")
qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever(), chain_type="stuff")

# 5. ❓ Ask something!
question = "Summarize the paper in simple terms."
answer = qa.run(question)
print("Q:", question)
print("A:", answer)


In [ ]:
question = "tell us about rangpur"
answer = qa.run(question)
print("Q:", question)
print("A:", answer)

In [ ]:
!nohup ollama serve > output.log 2>&1 &